<a href="https://colab.research.google.com/github/Otsemeuno/Customer-Subscription/blob/main/Customer_Subscription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Customer Subscription
$(Adapted from Kaggle)$


## Project Statement
This data is about a subscription-based digital product offering for financial advisory that includes newsletters, webinars, and investment recommendations. The offering has a couple of varieties, annual subscription, and digital subscription. The product also provides daytime support for customers to reach out to a care team that can help them with any product-related questions and signup/cancellation-related queries.

The data set contains the following information:

<ul><li>customer sign-up and cancellation dates at the product level
<li>call center activity
<li>customer demographics
<li>product pricing info<ul>

## Objectives

Forecast 2022 year revenue if the company expects to add the same number of new customers as the previous year

In 2022, one of the company's key initiatives is to improve the customer experience, one of the capabilities is superior customer care service, and they are thinking about enhancing in-product support better but that takes beyond 2022 to happen. It is very important to forecast call center case volume accurately so that they can make appropriate staffing decisions. Need to predict daily call volumes and make recommendations on how to staff on an hourly basis.

## Importing necessaries files

In [1]:
#importing drive
from google.colab import drive

In [62]:
#importing all necessary libraries
import pandas as pd
import numpy as np
from datetime import datetime

##Data Preparation

####Load the dataset

In [3]:
# Mount Google Drive on Kernel
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
customer_cases = pd.read_csv('/content/drive/MyDrive/Kaggle project/customer_cases.csv', index_col =[0])

In [5]:
customer_info = pd.read_csv('/content/drive/MyDrive/Kaggle project/customer_info.csv', index_col =[0])

In [6]:
customer_product = pd.read_csv('/content/drive/MyDrive/Kaggle project/customer_product.csv', index_col =[0])

In [7]:
product_info = pd.read_csv('/content/drive/MyDrive/Kaggle project/product_info.csv', index_col =[0])

In [8]:
#display top 5 rows in the dataset
customer_cases.head()

,case_id,date_time,customer_id,channel,reason
1,CC101,2017-01-01 10:32:03,C2448,phone,signup
2,CC102,2017-01-01 11:35:47,C2449,phone,signup
3,CC103,2017-01-01 11:37:09,C2450,phone,signup
4,CC104,2017-01-01 13:28:14,C2451,phone,signup
5,CC105,2017-01-01 13:52:22,C2452,phone,signup


In [9]:
#display top 5 rows in the dataset
customer_info.head()

,customer_id,age,gender
1,C2448,76,female
2,C2449,61,male
3,C2450,58,female
4,C2451,62,female
5,C2452,71,male


In [10]:
#display top 5 rows in the dataset
customer_product.head() 

,customer_id,product,signup_date_time,cancel_date_time
1,C2448,prd_1,2017-01-01 10:35:09,NaN
2,C2449,prd_1,2017-01-01 11:39:29,2021-09-05 10:00:02
3,C2450,prd_1,2017-01-01 11:42:00,2019-01-13 16:24:55
4,C2451,prd_2,2017-01-01 13:32:08,NaN
5,C2452,prd_1,2017-01-01 13:57:30,2021-06-28 18:06:01


In [11]:
#display top 5 rows in the dataset
product_info.head()

,name,price,billing_cycle
product_id,,,
prd_1,annual_subscription,1200,12
prd_2,monthly_subscription,125,1


In [12]:
customer_cases.shape

(330512, 5)

The customer cases data set has 5 attributes and 330512 observations

In [13]:
customer_info.shape

(508932, 3)

The data on customer info has 3 attributes and 508932 observations

In [14]:
customer_product.shape

(508932, 4)

Customer product has 508932 records and 4 features

In [15]:
product_info.shape

(2, 3)

Data provided has information on two products

###Missing Variables

In [16]:
customer_cases.isna().sum()

case_id        0
date_time      0
customer_id    0
channel        0
reason         0
dtype: int64

In [18]:
customer_info.isna().sum()

customer_id    0
age            0
gender         0
dtype: int64

In [19]:
customer_product.isna().sum()

customer_id              0
product                  0
signup_date_time         0
cancel_date_time    396447
customer                 0
dtype: int64

In [20]:
product_info.isna().sum()

name             0
price            0
billing_cycle    0
dtype: int64

Customer_produc has missing values. This missing values are from the cancel_date_time attribute. The values are missing because the customers are yet to cancel their current plan and are still with the company.

##Feature Engineering

In [17]:
customer_product['customer'] = customer_product.cancel_date_time.apply(lambda x: 1 if x is np.nan else 0)

In [ ]:
customer_product.columns

Index(['customer_id', 'product', 'signup_date_time', 'cancel_date_time',
       'customer', 'Customer'],
      dtype='object')

## Forecasting Revenue

Merge the two dataset to get all  the prices of each product subscribed by the customer.

In [21]:
df_rev = customer_product.merge(product_info, left_on = 'product', right_on = 'product_id')
df_rev = df_rev['signup_date_time','price']


,customer_id,product,signup_date_time,cancel_date_time,customer,name,price,billing_cycle
0,C2448,prd_1,2017-01-01 10:35:09,NaN,1,annual_subscription,1200,12
1,C2449,prd_1,2017-01-01 11:39:29,2021-09-05 10:00:02,0,annual_subscription,1200,12
2,C2450,prd_1,2017-01-01 11:42:00,2019-01-13 16:24:55,0,annual_subscription,1200,12
3,C2452,prd_1,2017-01-01 13:57:30,2021-06-28 18:06:01,0,annual_subscription,1200,12
4,C2453,prd_1,2017-01-01 15:26:14,NaN,1,annual_subscription,1200,12
...,...,...,...,...,...,...,...,...
508927,C511375,prd_2,2021-12-31 18:37:58,NaN,1,monthly_subscription,125,1
508928,C511376,prd_2,2021-12-31 18:42:00,NaN,1,monthly_subscription,125,1
508929,C511377,prd_2,2021-12-31 18:50:53,NaN,1,monthly_subscription,125,1
508930,C511378,prd_2,2021-12-31 19:00:38,NaN,1,monthly_subscription,125,1


  Replace the null values with the last day of the year 2021

In [47]:
#Convert the signup date and cancellation dates to datetime
df_rev['signup_date_time'] = pd.to_datetime(df_rev['signup_date_time']).dt.date
df_rev['cancel_date_time'] = pd.to_datetime(df_rev['cancel_date_time']).dt.date

In [49]:
# Define the proposed date
proposed_date = pd.to_datetime('2021-12-31').date

# Replace the NaT value with the proposed date
df_rev['cancel_date_time'] = df_rev['cancel_date_time'].fillna(proposed_date)

#get date from date time
df_rev['cancel_date_time'] = pd.to_datetime(df_rev['cancel_date_time']).dt.date



In [50]:
df_rev

,customer_id,product,signup_date_time,cancel_date_time,customer,name,price,billing_cycle
0,C2448,prd_1,2017-01-01,2021-12-31,1,annual_subscription,1200,12
1,C2449,prd_1,2017-01-01,2021-09-05,0,annual_subscription,1200,12
2,C2450,prd_1,2017-01-01,2019-01-13,0,annual_subscription,1200,12
3,C2452,prd_1,2017-01-01,2021-06-28,0,annual_subscription,1200,12
4,C2453,prd_1,2017-01-01,2021-12-31,1,annual_subscription,1200,12
...,...,...,...,...,...,...,...,...
508927,C511375,prd_2,2021-12-31,2021-12-31,1,monthly_subscription,125,1
508928,C511376,prd_2,2021-12-31,2021-12-31,1,monthly_subscription,125,1
508929,C511377,prd_2,2021-12-31,2021-12-31,1,monthly_subscription,125,1
508930,C511378,prd_2,2021-12-31,2021-12-31,1,monthly_subscription,125,1


Separate the data into monthly subscription and yearly subscription

In [59]:
df_rev_month = df_rev[df_rev['name'] == 'monthly_subscription']

In [80]:
df_rev_year = df_rev[df_rev['name'] == 'annual_subscription']
df_rev_year.head()


,customer_id,product,signup_date_time,cancel_date_time,customer,name,price,billing_cycle
0,C2448,prd_1,2017-01-01,2021-12-31,1,annual_subscription,1200,12
1,C2449,prd_1,2017-01-01,2021-09-05,0,annual_subscription,1200,12
2,C2450,prd_1,2017-01-01,2019-01-13,0,annual_subscription,1200,12
3,C2452,prd_1,2017-01-01,2021-06-28,0,annual_subscription,1200,12
4,C2453,prd_1,2017-01-01,2021-12-31,1,annual_subscription,1200,12


For the monthly subscription, we need to find the number of active months subscribed.

In [ ]:
df_rev_month.loc[:, 'months'] = df_rev_month.apply(lambda row: len(pd.period_range(start=row['signup_date_time'], end=row['cancel_date_time'], freq='M')), axis=1)

In [79]:
##indetify the months in the range of dataset
start_date = pd.to_datetime('2017-01-01')
end_date = pd.to_datetime('2021-12-31')
date_range = pd.date_range(start_date, end_date, freq = 'MS')
monthly_revenue = pd.DataFrame(index = date_range)
monthly_revenue


""
2017-01-01
2017-02-01
2017-03-01
2017-04-01
2017-05-01
2017-06-01
2017-07-01
2017-08-01
2017-09-01
2017-10-01
